# MIE 1622H: Assignment 1 - Mean-Variance Portfolio Selection Strategies

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import math
import cplex 
import matplotlib.pyplot as plt

## 1. Implement investment strategies in Python
##### (1) “Buy and hold" strategy
hold initial portfolio for the entire investment horizon of 2 years 

In [2]:
def strat_buy_and_hold(x_init, cash_init, mu, Q, cur_prices):
    x_optimal = x_init
    cash_optimal = cash_init
    return x_optimal, cash_optimal

##### (2) "Equally weighted" portfolio strategy
asset weights are selected as $w^t_i=\frac{1}{n}$, where n is the number of assets. You may need to re-balance
your portfolio in each period as the number of shares $x^t_i$ changes even when $w^t_i=\frac{1}{n}$
stays the same in each period.  

In [3]:
def strat_equally_weighted(x_init, cash_init, mu, Q, cur_prices):
    # Input) x_init: initial position/ cash_init: cash account before portfolio re-balancing
    # Output) x_optimal: current position/ cash_optimal: cash account after portfolio re-balancing

    portfolio_value = np.dot(x_init,cur_prices) + cash_init 
    asset_value = portfolio_value / N
    x_optimal = asset_value / cur_prices
    
    transaction_cost = np.dot(abs(x_optimal-x_init),cur_prices) * 0.005
    # new cash account  = value sold + old cash account - value bought - transaction cost
    cash_optimal = portfolio_value - np.sum(x_optimal * cur_prices) - transaction_cost  
   
    # rounding procedure 
    if cash_optimal >= 0:
        x_optimal = np.ceil(asset_value / cur_prices)
        transaction_cost = np.dot(abs(x_optimal-x_init),cur_prices) * 0.005
        cash_optimal = portfolio_value - np.sum(x_optimal * cur_prices) - transaction_cost 
    
    while cash_optimal < 0: #(cash account must be nonnegative)  
        # update x_optimal, transaction_cost & cash_optimal
        x_optimal = np.array(list(map(lambda x: np.ceil(x * (1-0.0005)), x_optimal)))
        transaction_cost = np.dot(abs(x_optimal-x_init),cur_prices) * 0.005
        cash_optimal = portfolio_value - np.sum(x_optimal * cur_prices) - transaction_cost 
  
    return x_optimal, cash_optimal

##### (3) "Minimum variance" portfolio strategy 
compute minimum variance portfolio for each period and re-balance accordingly.

\begin{equation}
\begin{array}{rl}
\displaystyle \min_{w} & w^TQw \\
s.t.&\sum_i w_i = 1\\
& w \geq 0
\end{array}
\end{equation}

In [4]:
def strat_min_variance(x_init, cash_init, mu, Q, cur_prices):
    
    cpx = cplex.Cplex()
    # Disable CPLEX output to screen
    cpx.set_results_stream(None)
    cpx.set_warning_stream(None)
    cpx.objective.set_sense(cpx.objective.sense.minimize)
    
    c  = [0.0] * N # no linear part of objectives function 
    lb = [0.0] * N # lower bounds on variables
    ub = [np.inf] * N # upper bounds on variables
    
    # constrian matrix A
    A = []
    for k in range(N):
        A.append([[0],[1.0]])  
    
    var_names = ["w_%s" % i for i in range(1,N+1)]
    
    # Add objective function, bounds on variables and constrains to CPLEX model
    cpx.linear_constraints.add(rhs=[1], senses="E")
    cpx.variables.add(obj=c, lb=lb, ub=ub, columns=A, names=var_names)
    
    # define and add quadratic part of objective function 
    Qmat = [[list(range(N)), list(2*Q[k,:])] for k in range(N)]
    cpx.objective.set_quadratic(Qmat)
    
    # solve optimal portfolio weights
    cpx.solve()
    w = np.array(cpx.solution.get_values())
    
    portfolio_value = np.dot(x_init,cur_prices) 
    x_optimal = portfolio_value * w / cur_prices
    transaction_cost = np.dot(abs(x_optimal-x_init),cur_prices) * 0.005
    cash_optimal = portfolio_value + cash_init - np.sum(x_optimal * cur_prices) - transaction_cost 
    
    # rounding procedure     
    if cash_optimal >= 0:
        x_optimal = np.ceil(x_optimal)
        transaction_cost = np.dot(abs(x_optimal-x_init),cur_prices) * 0.005
        cash_optimal = portfolio_value - np.sum(x_optimal * cur_prices) - transaction_cost 
        
    while cash_optimal < 0: #(cash account must be nonnegative)  
        # update x_optimal, transaction_cost & cash_optimal
        x_optimal = np.array(list(map(lambda x: np.ceil(x * (1-0.0005)), x_optimal)))
        transaction_cost = np.dot(abs(x_optimal-x_init),cur_prices) * 0.005
        cash_optimal = portfolio_value - np.sum(x_optimal * cur_prices) - transaction_cost 
  
    return x_optimal, cash_optimal

##### 4. "Maximum Sharpe ratio" portfolio strategy
compute a portfolio that maximizes Sharpe ratio for each period and re-balance accordingly

\begin{equation}
\begin{array}{rl}
\displaystyle \min_{y \in R^n, k \in R} & y^TQy \\
s.t. &\sum_i (\mu_i - rf)y_i = 1\\
&\sum_i y_i = k\\
& lk \leq Ay \leq uk\\
& k \geq 0
\end{array}
\end{equation}

In [5]:
daily_rf = 0.025/252

In [6]:
def strat_max_Sharpe(x_init, cash_init, mu, Q, cur_prices):
    
    cpx = cplex.Cplex()
    # Disable CPLEX output to screen
    cpx.set_results_stream(None)
    cpx.set_warning_stream(None)
    cpx.objective.set_sense(cpx.objective.sense.minimize)
    
    c  = [0.0] * (N+1) # no linear part of objectives function 
    lb = [0.0] * (N+1) # lower bounds on variables
    ub = [np.inf] * (N+1) # upper bounds on variables
    
    # constrian matrix A
    A = []
    for k in range(N):
        A.append([[0,1],[(mu - daily_rf)[k]]+[1]])  
    A.append([[0,1],[0,-1]])

    var_names = ["y_%s" % i for i in range(1,N+1)] + ["k"]
    
    # Add objective function, bounds on variables and constrains to CPLEX model
    cpx.linear_constraints.add(rhs=[1,0], senses="EE")
    cpx.variables.add(obj=c, lb=lb, ub=ub, columns=A, names=var_names)
    
    # define and add quadratic part of objective function 
    Qmat = [[list(range(N+1)), list(2*Q[k,:])+ [0]] for k in range(N)]
    Qmat.append([list(range(N+1)), list(21 *[0])])
    cpx.objective.set_quadratic(Qmat)
    
    # solve optimal portfolio weights
    cpx.solve()
    x = np.array(cpx.solution.get_values())
    
    w = np.array(np.divide(x[0:N],x[N]))
    
    portfolio_value = np.dot(x_init,cur_prices) 
    x_optimal = portfolio_value * w / cur_prices
    transaction_cost = np.dot(abs(x_optimal-x_init),cur_prices) * 0.005
    cash_optimal = portfolio_value + cash_init - np.sum(x_optimal * cur_prices) - transaction_cost 
      
    # rounding procedure  
    if cash_optimal >= 0:
        x_optimal = np.ceil(x_optimal)
        transaction_cost = np.dot(abs(x_optimal-x_init),cur_prices) * 0.005
        cash_optimal = portfolio_value - np.sum(x_optimal * cur_prices) - transaction_cost 
        
    while cash_optimal < 0: #(cash account must be nonnegative)  
        # update x_optimal, transaction_cost & cash_optimal
        x_optimal = np.array(list(map(lambda x: np.ceil(x * (1-0.002)), x_optimal)))
        transaction_cost = np.dot(abs(x_optimal-x_init),cur_prices) * 0.005
        cash_optimal = portfolio_value - np.sum(x_optimal * cur_prices) - transaction_cost

    return x_optimal, cash_optimal

##### 5. Try different variations of portfolio strategies 

In [7]:
def new_buy_and_hold(x_init, cash_init, mu, Q, cur_prices):
    # Input) x_init: initial position/ cash_init: cash account before portfolio re-balancing
    # Output) x_optimal: current position/ cash_optimal: cash account after portfolio re-balancing

    portfolio_value = np.dot(x_init,cur_prices) + cash_init 
    x_optimal =  (1000000/20) / data_prices[0]
    transaction_cost = np.dot(abs(x_optimal-x_init),cur_prices) * 0.005
    # new cash account  = value sold + old cash account - value bought - transaction cost
    cash_optimal = portfolio_value - np.sum(x_optimal * cur_prices) - transaction_cost  
   
    # rounding procedure    
    if cash_optimal >= 0:
        x_optimal = np.ceil(x_optimal)
        transaction_cost = np.dot(abs(x_optimal-x_init),cur_prices) * 0.005
        cash_optimal = portfolio_value - np.sum(x_optimal * cur_prices) - transaction_cost 
    
    while cash_optimal < 0: #(cash account must be nonnegative)  
        # update x_optimal, transaction_cost & cash_optimal
        x_optimal = np.array(list(map(lambda x: np.ceil(x * (1-0.0005)), x_optimal)))
        transaction_cost = np.dot(abs(x_optimal-x_init),cur_prices) * 0.005
        cash_optimal = portfolio_value - np.sum(x_optimal * cur_prices) - transaction_cost
        
    return x_optimal, cash_optimal

## 2. Analyze your results

In [8]:
# Input file
input_file_prices = 'Daily_closing_prices.csv'

# Read data into a dataframe
df = pd.read_csv(input_file_prices)
df

# Convert dates into array [year month day]
def convert_date_to_array(datestr):
    temp = [int(x) for x in datestr.split('/')]
    return [temp[-1], temp[0], temp[1]]

dates_array = np.array(list(df['Date'].apply(convert_date_to_array)))
data_prices = df.iloc[:, 1:].to_numpy()
dates = np.array(df['Date'])

# Find the number of trading days in Nov-Dec 2018 and compute expected return and covariance matrix for period 1
day_ind_start0 = 0
day_ind_end0 = len(np.where(dates_array[:,0]==2018)[0])
cur_returns0 = data_prices[day_ind_start0+1:day_ind_end0,:] / data_prices[day_ind_start0:day_ind_end0-1,:] - 1
mu = np.mean(cur_returns0, axis = 0)
Q = np.cov(cur_returns0.T)

# Remove datapoints for year 2018
data_prices = data_prices[day_ind_end0:,:]
dates_array = dates_array[day_ind_end0:,:]
dates = dates[day_ind_end0:]

# Initial positions in the portfolio
init_positions = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 980, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20000])

# Initial value of the portfolio
init_value = np.dot(data_prices[0,:], init_positions)
print('\nInitial portfolio value = $ {}\n'.format(round(init_value, 2)))

# Initial portfolio weights
w_init = (data_prices[0,:] * init_positions) / init_value

# Number of periods, assets, trading days
N_periods = 6*len(np.unique(dates_array[:,0])) # 6 periods per year
N = len(df.columns)-1
N_days = len(dates)

# Annual risk-free rate for years 2019-2020 is 2.5%
r_rf = 0.025


Initial portfolio value = $ 1000070.06



In [9]:
# Number of strategies
strategy_functions = ['strat_buy_and_hold', 'strat_equally_weighted', 'strat_min_variance', 'strat_max_Sharpe','new buy and hold']
strategy_names     = ['Buy and Hold', 'Equally Weighted Portfolio', 'Minimum Variance Portfolio', 'Maximum Sharpe Ratio Portfolio','new buy and hold']
N_strat = len(strategy_functions)  # uncomment this in your code
fh_array = [strat_buy_and_hold, strat_equally_weighted, strat_min_variance, strat_max_Sharpe, new_buy_and_hold]

In [10]:
portf_value = [0] * N_strat
x = np.zeros((N_strat, N_periods),  dtype=np.ndarray)
cash = np.zeros((N_strat, N_periods),  dtype=np.ndarray)


for period in range(1, N_periods+1):
    # Compute current year and month, first and last day of the period
    if dates_array[0, 0] == 19:
        cur_year  = 19 + math.floor(period/7)
    else:
        cur_year  = 2019 + math.floor(period/7)

    cur_month = 2*((period-1)%6) + 1
    day_ind_start = min([i for i, val in enumerate((dates_array[:,0] == cur_year) & (dates_array[:,1] == cur_month)) if val])
    day_ind_end = max([i for i, val in enumerate((dates_array[:,0] == cur_year) & (dates_array[:,1] == cur_month+1)) if val])
    print('\nPeriod {0}: start date {1}, end date {2}'.format(period, dates[day_ind_start], dates[day_ind_end]))
   
    # Prices for the current day
    cur_prices = data_prices[day_ind_start,:]

    # Execute portfolio selection strategies
    for strategy in range(N_strat):
        # Get current portfolio positions
        if period == 1:
            curr_positions = init_positions
            curr_cash = 0
            portf_value[strategy] = np.zeros((N_days, 1))
        else:
            curr_positions = x[strategy, period-2]
            curr_cash = cash[strategy, period-2]

        # Compute strategy
        x[strategy, period-1], cash[strategy, period-1] = fh_array[strategy](curr_positions, curr_cash, mu, Q, cur_prices)

        # Verify that strategy is feasible (you have enough budget to re-balance portfolio)
        # Check that cash account is >= 0  
        # Check that we can buy new portfolio subject to transaction costs

        ###################### Insert your code here ############################
        if cash[strategy, period-1] < 0:
            print ('Strategy "{0}", cash account is negative'.format( strategy_names[strategy]))

        # Compute portfolio value
        p_values = np.dot(data_prices[day_ind_start:day_ind_end+1,:], x[strategy, period-1]) + cash[strategy, period-1]
        portf_value[strategy][day_ind_start:day_ind_end+1] = np.reshape(p_values, (p_values.size,1))
        print('Strategy "{0}", value begin = $ {1:.2f}, value end = $ {2:.2f}'.format( strategy_names[strategy], 
              portf_value[strategy][day_ind_start][0], portf_value[strategy][day_ind_end][0]))
      
    # Compute expected returns and covariances for the next period
    cur_returns = data_prices[day_ind_start+1:day_ind_end+1,:] / data_prices[day_ind_start:day_ind_end,:] - 1
    mu = np.mean(cur_returns, axis = 0)
    Q = np.cov(cur_returns.T)
 


Period 1: start date 01/02/2019, end date 02/28/2019
Strategy "Buy and Hold", value begin = $ 1000070.06, value end = $ 1121179.83
Strategy "Equally Weighted Portfolio", value begin = $ 991114.55, value end = $ 1097016.56
Strategy "Minimum Variance Portfolio", value begin = $ 991691.72, value end = $ 1057825.46
Strategy "Maximum Sharpe Ratio Portfolio", value begin = $ 990122.24, value end = $ 1016707.93
Strategy "new buy and hold", value begin = $ 991114.64, value end = $ 1097015.62

Period 2: start date 03/01/2019, end date 04/30/2019
Strategy "Buy and Hold", value begin = $ 1126131.27, value end = $ 1075001.89
Strategy "Equally Weighted Portfolio", value begin = $ 1103380.21, value end = $ 1188831.89
Strategy "Minimum Variance Portfolio", value begin = $ 1055521.17, value end = $ 1108271.91
Strategy "Maximum Sharpe Ratio Portfolio", value begin = $ 1007073.46, value end = $ 1076734.88
Strategy "new buy and hold", value begin = $ 1103662.39, value end = $ 1189909.60

Period 3: start

##### Plot one chart in Python that illustrates the daily value of your portfolio (for each trading strategy) over the years 2019 and 2020 using daily prices provided. Include the chart in your report.  

In [ ]:
# Plot results
###################### Insert your code here ############################

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(pd.to_datetime(dates),portf_value[0], label ='Buy and Hold')
ax.plot(pd.to_datetime(dates),portf_value[1], label ='Equally Weighted Portfolio')
ax.plot(pd.to_datetime(dates),portf_value[2], label ='Mininum Variance Portfolio')
ax.plot(pd.to_datetime(dates),portf_value[3], label ='Maximum Sharpe Ratio Portfolio')

plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.title('Portfolio Daily Value (Price)')
plt.legend()
plt.show()

##### Plot two charts in Python for strategy 3 and 4 to show dynamic changes in portfolio allocations. In each chart, x-axis represents the rolling up time horizon, y-axis denotes portfolio weights between 0 and 1, and distinct lines display the position of selected assets over time periods. You may use these figures to support your analysis or discussion.
 

In [ ]:
rebalance_date_index = [list(dates).index(i) for i in 
                        ['01/02/2019','03/01/2019','05/01/2019','07/01/2019','09/03/2019','11/01/2019',
                         '01/02/2020','03/02/2020','05/01/2020','07/01/2020','09/01/2020','11/02/2020']]
assets = list(df.columns[1:])

In [ ]:
Mininum_Variance = 2

fig, ax = plt.subplots(figsize=(12, 6))
allocation = []
for i in range(12):
    rebalance_position = x[Mininum_Variance][i]
    rebalance_price = data_prices[rebalance_date_index[i]]
    rebalance_portfolio_value = np.dot(rebalance_price,rebalance_position)
    allocation.append(list(rebalance_position * rebalance_price / rebalance_portfolio_value))

allocation = pd.DataFrame(allocation)
allocation.index = np.arange(1,13)

for i in range(20):
    plt.plot(allocation[i],label=assets[i-1])

plt.xlabel('Period')
plt.ylim(0,1)
plt.ylabel('Portfolio allocation')
plt.title("Mininum Variance Portfolio Allocation")
plt.legend(loc='upper right')
plt.show()

In [ ]:
Maximum_Sharpe = 3

fig, ax = plt.subplots(figsize=(12, 6))
allocation = []
for i in range(12):
    rebalance_position = x[Maximum_Sharpe][i]
    rebalance_price = data_prices[rebalance_date_index[i]]
    rebalance_portfolio_value = np.dot(rebalance_price,rebalance_position)
    allocation.append(list(rebalance_position * rebalance_price / rebalance_portfolio_value))

allocation = pd.DataFrame(allocation)
allocation.index = np.arange(1,13)

for i in range(20):
    plt.plot(allocation[i],label=assets[i-1])

plt.xlabel('Period')
plt.ylabel('Portfolio allocation')
plt.title("Maximum Sharpe Ratio Portfolio Allocation")
plt.legend(loc='upper right')
plt.show()

### 3. (10 %) Discuss possible improvements to your trading strategies:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(pd.to_datetime(dates),portf_value[0], label ='Buy and Hold')
ax.plot(pd.to_datetime(dates),portf_value[1], label ='Equally Weighted Portfolio')
ax.plot(pd.to_datetime(dates),portf_value[2], label ='Mininum Variance Portfolio')
ax.plot(pd.to_datetime(dates),portf_value[3], label ='Maximum Sharpe Ratio Portfolio')
ax.plot(pd.to_datetime(dates),portf_value[4], label ='new buy and hold')

plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.title('Portfolio Daily Value (Price)')
plt.legend()
plt.show()